In [ ]:
from google.cloud import bigquery
project_id = "steady-service-448416-k2"

dataset = "finance_economics_events_stg"
region = "us-central1"

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset steady-service-448416-k2.finance_economics_events_stg


Table | Column alter | Reason | category

state_exp_data_1991_2024| year | STRING -> FLOAT | 5

crime_data | year | INTEGER -> FLOAT | 5

crime_data | population | 0.0 -> Null | 6

gdp | year | quarters - > decimals | 5

mortality_and_natality | year | INTEGER -> FLOAT | 5

state_energy_consumption| year | INTEGER -> FLOAT | 5

state_energy_consumption | sector | sector -> primary_sector | 7




====================================


Those tables are alter to be more descriptive

state_exp_data_1991_2024_key | column_code -> the_abbreviation

state_exp_data_1991_2024_key | column_description - > the_full_name

state_level_unemployment_data | period -> year

us_geocode | stateandterritory -> state_abbreviation

us_geocode | name -> state_full_name


# state_exp_data_1991_2024

staging dataset change STRING to FLOAT

In [52]:
%%bigquery
SELECT
  SAFE_CAST(year AS FLOAT64) AS year,  -- Convert year from STRING to FLOAT64 safely
  * EXCEPT(year)  -- Keep all other columns unchanged
FROM finance_economics_events_raw.state_exp_data_1991_2024;


Query is running:   0%|          |

Downloading:   0%|          |

,year,state,elsed_gf,elsed_ff,elsed_of,elsed_bf,elsed_tot,hgred_gf,hgred_ff,hgred_of,...,oftot_capi,bftot_capi,total_capi,gftot_cap,fftot_cap,oftot_cap,bftot_cap,total_cap,_data_source,_load_time
0,2024.0,Alabama,6389.0,4386.0,230.0,291.0,11296.0,2600.0,1624.0,3334.0,...,8789.0,607.0,39201.0,5.0,346.0,583.0,607.0,1541.0,State Exp Report,2025-02-07 22:39:38.772182+00:00
1,2016.0,Alabama,4157.0,1008.0,189.0,58.0,5412.0,1493.0,1175.0,2634.0,...,7875.0,332.0,25866.0,7.0,272.0,339.0,332.0,950.0,State Exp Report,2025-02-07 22:39:38.772182+00:00
2,2021.0,Alabama,5012.0,974.0,222.0,134.0,6342.0,1824.0,1267.0,3189.0,...,8945.0,329.0,31914.0,14.0,245.0,429.0,329.0,1017.0,State Exp Report,2025-02-07 22:39:38.772182+00:00
3,2020.0,Alabama,4882.0,866.0,212.0,41.0,6001.0,1770.0,1286.0,3215.0,...,9056.0,217.0,31511.0,9.0,277.0,381.0,217.0,884.0,State Exp Report,2025-02-07 22:39:38.772182+00:00
4,2019.0,Alabama,4565.0,1049.0,405.0,24.0,6043.0,1655.0,1129.0,3137.0,...,8835.0,364.0,28886.0,9.0,281.0,322.0,364.0,976.0,State Exp Report,2025-02-07 22:39:38.772182+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,2016.0,Wyoming,1.0,0.0,806.0,0.0,807.0,355.0,1.0,7.0,...,1437.0,0.0,4102.0,67.0,33.0,39.0,0.0,139.0,State Exp Report,2025-02-07 22:39:38.772182+00:00
1814,2020.0,Wyoming,0.0,0.0,853.0,0.0,853.0,313.0,1.0,0.0,...,2419.0,0.0,4483.0,0.0,0.0,0.0,0.0,0.0,State Exp Report,2025-02-07 22:39:38.772182+00:00
1815,2021.0,Wyoming,0.0,0.0,913.0,0.0,913.0,318.0,2.0,48.0,...,3463.0,0.0,5620.0,1.0,0.0,1.0,0.0,2.0,State Exp Report,2025-02-07 22:39:38.772182+00:00
1816,2023.0,Wyoming,0.0,0.0,912.0,0.0,912.0,324.0,0.0,16.0,...,2831.0,0.0,5353.0,70.0,0.0,78.0,0.0,148.0,State Exp Report,2025-02-07 22:39:38.772182+00:00


Create staging table

In [51]:
%%bigquery
CREATE OR REPLACE TABLE finance_economics_events_stg.state_exp_data_1991_2024 AS
SELECT
  SAFE_CAST(year AS FLOAT64) AS year,
  * EXCEPT(year)
FROM finance_economics_events_raw.state_exp_data_1991_2024;


Query is running:   0%|          |

""


# crime_data

staging dataset
Replace 0 with null

In [50]:
%%bigquery
SELECT
  CAST(year AS FLOAT64) AS year,  -- Convert year to FLOAT
  CASE WHEN population = 0.0 THEN NULL ELSE population END AS population,  -- Handle population values
  * EXCEPT(year, population)  -- Avoid duplication
FROM finance_economics_events_raw.crime_data;







Query is running:   0%|          |

Downloading:   0%|          |

,year,population,state,crime,crime_against_persons,crime_against_property,crime_against_society,_data_source,_load_time
0,2020.0,12504.0,Pennyslvania,156.0,22.0,98.0,36.0,CrimeData,2025-02-07 03:06:20.701827+00:00
1,2020.0,12504.0,Pennyslvania,156.0,22.0,98.0,36.0,CrimeData,2025-02-07 03:07:32.577832+00:00
2,2020.0,35257.0,Wyoming,2110.0,693.0,973.0,444.0,CrimeData,2025-02-07 03:07:32.577832+00:00
3,2020.0,35257.0,Wyoming,2110.0,693.0,973.0,444.0,CrimeData,2025-02-07 03:06:20.701827+00:00
4,2020.0,NaN,District_of_Columbia,2535.0,1062.0,1065.0,408.0,CrimeData,2025-02-07 03:07:32.577832+00:00
...,...,...,...,...,...,...,...,...,...
397,2023.0,20415931.0,California,1169699.0,256706.0,724983.0,188010.0,CrimeData,2025-02-07 03:06:20.701827+00:00
398,2023.0,11382236.0,New_York,781651.0,282634.0,449617.0,49400.0,CrimeData,2025-02-07 03:06:20.701827+00:00
399,2023.0,11382236.0,New_York,781651.0,282634.0,449617.0,49400.0,CrimeData,2025-02-07 03:07:32.577832+00:00
400,2023.0,22594211.0,Texas,1683859.0,463804.0,996810.0,223245.0,CrimeData,2025-02-07 03:07:32.577832+00:00


Create staging table

In [49]:
%%bigquery
CREATE OR REPLACE TABLE finance_economics_events_stg.crime_data AS
SELECT
  CAST(year AS FLOAT64) AS year,
  CASE WHEN population = 0.0 THEN NULL ELSE population END AS population,
  * EXCEPT(year, population)
FROM finance_economics_events_raw.crime_data;


Query is running:   0%|          |

""


# mortality_and_natality

staging dataset, change STRING to FLOAT

In [44]:
%%bigquery
SELECT
  CAST(year AS FLOAT64) AS year,
  * EXCEPT(year)
FROM finance_economics_events_raw.mortality_and_natality;


Query is running:   0%|          |

Downloading:   0%|          |

,year,state,month,period,indicator,data_value,_data_source,_load_time
0,2023.0,ALABAMA,April,Monthly,Number of Live Births,4391,health,2025-02-07 03:06:04.897834+00:00
1,2023.0,ALABAMA,April,Monthly,Number of Live Births,4391,health,2025-02-07 03:07:19.276960+00:00
2,2023.0,ALABAMA,February,Monthly,Number of Live Births,4420,health,2025-02-07 03:06:04.897834+00:00
3,2023.0,ALABAMA,February,Monthly,Number of Live Births,4420,health,2025-02-07 03:07:19.276960+00:00
4,2024.0,ALABAMA,February,Monthly,Number of Live Births,4485,health,2025-02-07 03:07:19.276960+00:00
...,...,...,...,...,...,...,...,...
3955,2023.0,WYOMING,August,Monthly,Number of Live Births,538,health,2025-02-07 03:07:19.276960+00:00
3956,2023.0,WYOMING,June,Monthly,Number of Live Births,545,health,2025-02-07 03:07:19.276960+00:00
3957,2023.0,WYOMING,June,Monthly,Number of Live Births,545,health,2025-02-07 03:06:04.897834+00:00
3958,2024.0,WYOMING,January,Monthly,Number of Deaths,552,health,2025-02-07 03:07:19.276960+00:00


create staging table

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE finance_economics_events_stg.mortality_and_natality AS
SELECT
  CAST(year AS FLOAT64) AS year,
  * EXCEPT(year)
FROM finance_economics_events_raw.mortality_and_natality;


#gdp


staging dataset, change STRING to FLOAT

In [21]:
%%bigquery
SELECT
  SAFE_CAST(
    REGEXP_REPLACE(
      REGEXP_REPLACE(
        REGEXP_REPLACE(
          REGEXP_REPLACE(
            year, r'q1[^0-9]*', '.25'),
          r'q2[^0-9]*', '.50'),
        r'q3[^0-9]*', '.75'),
      r'q4[^0-9]*', '.99')
    AS FLOAT64
  ) AS year
FROM finance_economics_events_raw.gdp;


Query is running:   0%|          |

Downloading:   0%|          |

,year
0,2022.00
1,2022.00
2,2023.00
3,2023.00
4,2024.25
...,...
1695,2024.25
1696,2024.50
1697,2024.50
1698,2024.75


create staging table

In [23]:
# 2024 -> 2024.0
# 2024q1 -> 2024.25
# 2024q2 -> 2024.50
# 2024q3 -> 2024.75
# 2024q4 -> 2024.99

%%bigquery
create or replace table finance_economics_events_stg.gdp as
  select
    SAFE_CAST(
    REGEXP_REPLACE(
      REGEXP_REPLACE(
        REGEXP_REPLACE(
          REGEXP_REPLACE(
            year, r'q1[^0-9]*', '.25'),
          r'q2[^0-9]*', '.50'),
        r'q3[^0-9]*', '.75'),
      r'q4[^0-9]*', '.99')
    AS FLOAT64
  ) AS year,
  *
  except(year)
  from finance_economics_events_raw.gdp;

Query is running:   0%|          |

""


# state_energy_consumption

stage the dataset, only show the primary sector if multipule sectors are found

In [27]:
%%bigquery
SELECT
  COALESCE(REGEXP_EXTRACT(sector, r'^(.*?)\s+and'), sector) AS primary_sector
FROM finance_economics_events_raw.state_energy_consumption;


Query is running:   0%|          |

Downloading:   0%|          |

,primary_sector
0,Coal generating units capacity factor
1,Coal generating units capacity factor
2,Coal generating units capacity factor
3,Coal generating units capacity factor
4,Coal generating units capacity factor
...,...
403,Total end-use sector energy consumption
404,Total end-use sector energy consumption
405,Total end-use sector energy consumption
406,Total end-use sector energy consumption


create staging table

In [30]:
%%bigquery
create or replace table finance_economics_events_stg.state_energy_consumption as
select
  COALESCE(REGEXP_EXTRACT(sector, r'^(.*?)\s+and'), sector) AS primary_sector,
  *
  except(sector)
from finance_economics_events_raw.state_energy_consumption;

Query is running:   0%|          |

""


create staging table

In [46]:
%%bigquery
CREATE OR REPLACE TABLE finance_economics_events_stg.state_energy_consumption AS
SELECT
  CAST(year AS FLOAT64) AS year,
  * EXCEPT(year)
FROM finance_economics_events_raw.state_energy_consumption;


Query is running:   0%|          |

""


# Change some tables to be more descriptive

state_exp_data_1991_2024_key

In [42]:
%%bigquery
CREATE OR REPLACE TABLE finance_economics_events_stg.state_exp_data_1991_2024_key AS
WITH NumberedRows AS (
  SELECT *, ROW_NUMBER() OVER () AS row_num
  FROM finance_economics_events_raw.state_exp_data_1991_2024_key
)
SELECT
  column_code AS the_abbreviation,
  column_description AS the_full_name,
  * EXCEPT(row_num, column_code, column_description)
FROM NumberedRows
WHERE row_num >= 1734;


Query is running:   0%|          |

""


state_level_unemployment_data

In [45]:
%%bigquery
CREATE OR REPLACE TABLE finance_economics_events_stg.state_level_unemployment_data AS
SELECT
  CAST(period AS FLOAT64) AS year,  -- Convert period to FLOAT and rename to year
  * EXCEPT(period)  -- Keep all other columns unchanged
FROM finance_economics_events_raw.state_level_unemployment_data;


Query is running:   0%|          |

""


us_geocode

In [54]:
%%bigquery
CREATE OR REPLACE TABLE finance_economics_events_stg.us_geocode AS
SELECT
  stateandterritory AS state_abbreviation,  -- Rename stateandterritory
  TRIM(name) AS state_full_name,  -- Remove leading spaces
  * EXCEPT(stateandterritory, name)  -- Keep all other columns unchanged
FROM finance_economics_events_raw.us_geocode;


Query is running:   0%|          |

""
